In [106]:
import numpy as np
from svgpathtools import svg2paths, Path, Line, wsvg
import svgpathtools
from rdp import rdp
from svg.path import parse_path
import os
import pandas as pd

In [125]:
# define paths
path_to_svg = './svg'
save_dir = './spline_coords'

In [97]:
def read_svg_file(filename):
    paths, attributes =  svg2paths(filename)
    return paths

In [109]:
all_classes = [i for i in os.listdir(path_to_svg) if os.path.isdir(os.path.join(path_to_svg,i))]
assert len(all_classes)==125
# loop through classes
for c in all_classes:
    class_path = os.path.join(path_to_svg,c)
    all_sketches = [i for i in os.listdir(class_path) if i.split('.')[1]=='svg']
    
    # initialize
    start_x = []
    start_y = []
    c1_x = []
    c1_y = []
    c2_x = []
    c2_y = []
    end_x = []
    end_y = []
    stroke_num = []
    spline_num = []
    stroke_counter = 0
    spline_counter = 0

    class_name = []
    photo_name = []
    sketch_name = []

    path_to_invalid = os.path.join(class_path,'invalid.txt')
    with open(path_to_invalid) as f:
        invalid = set([line[:-2] for line in f.readlines() if line[0]=='n'])
    cat = class_path.split('/')[-1]
    # loop through sketches
    for i,s in enumerate(all_sketches):
        if i%100==0:
            print 'Extracted {} of {} {} sketches...'.format(i, len(all_sketches), cat)
        if s in invalid:
            print s + ' marked invalid, moving on...'
        else:
            sketch_path = os.path.join(path_to_svg,c,s)
            # read in paths and loop through to get strokes and spline segments
            paths = read_svg_file(sketch_path)        
            for path in paths:
                stroke_counter += 1
                for point in path:
                    if type(point) == svgpathtools.path.CubicBezier:
                        start_x.append(np.real(point.start))
                        start_y.append(np.imag(point.start))   
                        c1_x.append(np.real(point.control1))
                        c1_y.append(np.imag(point.control1))
                        c2_x.append(np.real(point.control2))
                        c2_y.append(np.imag(point.control2))
                        end_x.append(np.real(point.end))
                        end_y.append(np.imag(point.end))
                        spline_num.append(spline_counter)
                        spline_counter += 1         
                        stroke_num.append(stroke_counter)
                        class_name.append(os.path.dirname(sketch_path).split('/')[-1])
                        photo_name.append(os.path.basename(sketch_path).split('-')[0] + '.jpg')
                        sketch_name.append(os.path.basename(sketch_path))                    

    df = pd.DataFrame([start_x,start_y,c1_x,c1_y,c2_x,c2_y,end_x,end_y, \
                       stroke_num,spline_num,class_name,photo_name,sketch_name]) 
    df = df.transpose()
    df.columns = ['start_x','start_y','c1_x','c1_y','c2_x','c2_y','end_x','end_y', \
                  'stroke_num','spline_num','class_name','photo_name','sketch_name']

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    save_path = os.path.join(save_dir, cat + '.csv')
    df.to_csv(save_path)
    

Extracted 0 of 709 airplane sketches...
Extracted 100 of 709 airplane sketches...
Extracted 200 of 709 airplane sketches...
Extracted 300 of 709 airplane sketches...
Extracted 400 of 709 airplane sketches...
Extracted 500 of 709 airplane sketches...
Extracted 600 of 709 airplane sketches...
Extracted 700 of 709 airplane sketches...
